# Fixture Observation

#### This script creates a fixture observation for each match, combining the home and away teams statistics, as well as the prediction target - pre-match odds

In [7]:
import boto3
import pandas as pd
import numpy as np
import json
from io import StringIO


s3 = boto3.resource('s3')
bucket_name = 'betfairex'


def read_csv_from_s3(bucket_name, object_key):
    response = s3.Object(bucket_name, object_key).get()

    s3_contents = response['Body'].read()
    df = pd.read_csv(StringIO(s3_contents.decode('utf-8')))

    return df

In [8]:
bucket = s3.Bucket(bucket_name)

In [9]:
all_keys = [obj.key for obj in bucket.objects.all()]

# keys are in the regular expression as follows:
# YYYY-MM-DD_[HOME_TEAM] v [AWAY_TEAM].csv

In [19]:
LEAGUE_INFO = pd.read_csv('league_info.csv')
TEAM_INFO = pd.read_csv('team_info.csv')

def get_match_info(object_key):
    match_name = object_key[:-4]
    match_date_str = match_name.split('_')[0]
    fixture = match_name.split('_')[1]
    home_team = fixture.split(' v ')[0]
    away_team = fixture.split(' v ')[1]
    
    match_info = {
        'match_date': match_date_str,
        'home_team': home_team,
        'away_team': away_team,
    }    
    return match_info

def get_betfair_exchange_prematch_odds(bucket_name, object_key):
    match_info = get_match_info(object_key)
    
    odds_df = read_csv_from_s3(bucket_name, object_key)
    
    odds_df = odds_df.sort_values("timestamp", ascending=True)
    prematch_df = odds_df[~odds_df['inplay']]
    
    home_team_df = prematch_df[prematch_df['selection'] == match_info['home_team']]
    away_team_df = prematch_df[prematch_df['selection'] == match_info['away_team']]
    draw_df = prematch_df[prematch_df['selection'] == "Draw"]
    
    home_prematch_odds = home_team_df.tail(1)['odds'].iloc[0]
    away_prematch_odds = away_team_df.tail(1)['odds'].iloc[0]
    draw_prematch_odds = draw_df.tail(1)['odds'].iloc[0]
    
    prematch_odds = {
        'match_date': match_info['match_date'],
        match_info['home_team']: home_prematch_odds,
        match_info['away_team']: away_prematch_odds,
        'Draw': draw_prematch_odds
    }
    return prematch_odds    
    

In [43]:
# Configuration constants

SERVER = "v3.football.api-sports.io"
API_KEY = "1f5008c4f33481203631d90c7a81c5e5"

from pandas.tseries.offsets import Day
import http.client
import datetime as dt

SERVER = "v3.football.api-sports.io"



conn = http.client.HTTPSConnection(SERVER)

headers = {
  'x-rapidapi-host': SERVER,
  'x-rapidapi-key': API_KEY
}


def get_team_stats(season, league_id, team_id, as_of_date):
    # as_of_date in format YYYY-MM-DD
    conn.request("GET", "/teams/statistics?season=%s&league=%s&team=%s&date=%s" % (season, league_id, team_id, as_of_date), headers=headers)
    res = conn.getresponse()
    data = res.read()

    team_stats_raw = data.decode("utf-8")
    team_stats = json.loads(team_stats_raw)["response"]
    
    return team_stats
    
TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL = {
    "Athletic Bilbao": "Athletic Club"
}

def get_team_stats_for_fixture(object_key):
    match_info = get_match_info(object_key)
    
    match_date = match_info['match_date']
    match_date = dt.datetime.strptime(match_date, "%Y-%m-%d")
    as_of_date = match_date - Day(1)
    as_of_date = as_of_date.strftime("%Y-%m-%d")

    # Decide which season to query. In API-Football, season corresponds to the starting year. Eg. 24-25 season will be season 2024.
    match_year = match_date.year
    match_month = match_date.month
    
    if match_month < 7:
        season = match_year - 1
    else:
        season = match_year
    
    home_team = TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL.get(match_info['home_team'], match_info['home_team'])
    home_team_info = TEAM_INFO.query("name == @home_team").to_dict('records')[0]
    home_team_id = home_team_info['team_id']
    home_team_league_id = home_team_info['league_id']
    home_team_stats = get_team_stats(season, home_team_league_id, home_team_id, as_of_date)
    
    away_team = TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL.get(match_info['away_team'], match_info['away_team'])
    away_team_info = TEAM_INFO.query("name == @away_team").to_dict('records')[0]
    away_team_id = away_team_info['team_id']
    away_team_league_id = away_team_info['league_id']
    away_team_stats = get_team_stats(season, away_team_league_id, away_team_id, as_of_date)
    
    return home_team_stats, away_team_stats
    

In [44]:
home_team_stats, away_team_stats = get_team_stats_for_fixture(all_keys[-1])

In [47]:
def calc_form_score(form):
    form = form[-6:]
    form_dict = {
        'W': 1,
        'D': 0,
        'L': -1
    }
    
    form_score = 0
    for res in form[:3]:
        form_score += 0.5 * form_dict[res]
    for res in form[3:]:
        form_score += 1 * form_dict[res]
    return form_score

def generate_features(object_key):
    row = {}
    match_info = get_match_info(object_key)
    
    # Index
    row['match_date'] = match_info['match_date']
    row['home_team'] = match_info['home_team']
    row['away_team'] = match_info['away_team']
    
    # Features
    home_team_stats, away_team_stats = get_team_stats_for_fixture(object_key)
    row['home_team_avg_home_goals'] = home_team_stats['goals']['for']['average']['home']
    row['home_team_avg_total_goals'] = home_team_stats['goals']['for']['average']['total']
    row['away_team_avg_away_goals'] = away_team_stats['goals']['for']['average']['away']
    row['away_team_avg_total_goals'] = away_team_stats['goals']['for']['average']['total']
    row['home_team_form_score'] = calc_form_score(home_team_stats['form'])
    row['away_team_form_score'] = calc_form_score(away_team_stats['form'])
    
    # Labels
    prematch_odds = get_betfair_exchange_prematch_odds(bucket_name, object_key)
    row['odds_home_team'] = prematch_odds[match_info['home_team']]
    row['odds_away_team'] = prematch_odds[match_info['away_team']]
    row['odds_draw'] = prematch_odds['Draw']
    
    return row

In [49]:
from tqdm import tqdm
rows = []

for i in tqdm(range(len(all_keys))):
    key = all_keys[i]
    
    try:
        row = generate_features(key)
        rows.append(row)
    except:
        print("Failed to generate feature for %s." % key)

  1%|▎                                          | 3/469 [00:00<01:10,  6.56it/s]

Failed to generate feature for 2023-08-13_Celta Vigo v Osasuna.csv.
Failed to generate feature for 2023-08-13_Getafe v Barcelona.csv.


  2%|▋                                          | 8/469 [00:01<01:05,  7.01it/s]

Failed to generate feature for 2023-08-13_Villarreal v Betis.csv.
Failed to generate feature for 2023-08-20_Betis v Atletico Madrid.csv.


  4%|█▋                                        | 19/469 [00:03<01:13,  6.10it/s]

...

Failed to generate feature for 2024-12-14_Rayo Vallecano v Real Madrid.csv.
Failed to generate feature for 2024-12-14_Sevilla v Celta Vigo.csv.
Failed to generate feature for 2024-12-15_Alaves v Athletic Bilbao.csv.
Failed to generate feature for 2024-12-15_Atletico Madrid v Getafe.csv.


100%|████████████████████████████████████████▊| 467/469 [01:07<00:00, 17.57it/s]

Failed to generate feature for 2024-12-15_Barcelona v Leganes.csv.
Failed to generate feature for 2024-12-15_Real Sociedad v Las Palmas.csv.
Failed to generate feature for 2024-12-15_Villarreal v Betis.csv.
Failed to generate feature for 2024-12-18_Espanyol v Valencia.csv.


100%|█████████████████████████████████████████| 469/469 [01:08<00:00,  6.89it/s]

Failed to generate feature for 2024-12-18_Villarreal v Rayo Vallecano.csv.
Failed to generate feature for 2024-12-20_Girona v Valladolid.csv.


In [51]:
df = pd.DataFrame.from_records(rows)

In [52]:
df

,match_date,home_team,away_team,home_team_avg_home_goals,home_team_avg_total_goals,away_team_avg_away_goals,away_team_avg_total_goals,home_team_form_score,away_team_form_score,odds_home_team,odds_away_team,odds_draw
0,2023-08-13,Athletic Bilbao,Real Madrid,0.0,0.0,2.0,2.0,-0.5,0.5,1000.00,1.01,1000.00
1,2023-08-13,Las Palmas,Mallorca,1.0,1.0,1.0,1.0,0.0,0.0,300.00,110.00,1.01
2,2023-08-13,Real Sociedad,Girona,1.0,1.0,1.0,1.0,0.0,0.0,160.00,140.00,1.01
3,2023-08-13,Sevilla,Valencia,1.0,1.0,2.0,2.0,-0.5,0.5,1000.00,1.01,150.00
4,2023-08-20,Alaves,Sevilla,0.0,0.0,0.0,1.0,-0.5,-0.5,3.65,2.46,3.20
...,...,...,...,...,...,...,...,...,...,...,...,...
255,2024-08-23,Sevilla,Villarreal,0.0,2.0,0.0,2.0,0.0,0.0,2.32,3.35,3.70
256,2024-08-24,Barcelona,Athletic Bilbao,0.0,2.0,0.0,1.0,0.5,0.0,1.69,5.80,4.20
257,2024-08-24,Espanyol,Real Sociedad,0.0,0.0,0.0,1.0,-0.5,-0.5,4.00,2.30,3.15
258,2024-08-24,Getafe,Rayo Vallecano,0.0,1.0,2.0,2.0,0.0,0.5,2.76,3.25,3.00
